# ESCO Skills Preprocessing
Here, the target database to which the extracted phrases and units are to be matched is cleaned and embedded. Functions are called from the Utils module.

In [4]:
import json
import re
import sqlite3
from pathlib import Path
import numpy as np
import pandas as pd
import config as C
from utils import embed_texts, df_to_sqlite 

csv_path = Path(C.ESCO_CSV)
sqlite_out = Path(C.SQLITE_PATH)

print("Using ESCO CSV:", csv_path)
print("Using SQLite DB:", sqlite_out)

Using ESCO CSV: /Users/denysheryavenko/Desktop/EL Thesis/data/skills_de.csv
Using SQLite DB: /Users/denysheryavenko/Desktop/EL Thesis/data/db.sqlite


### Minimal cleaning
We don't do any crazy cleaning here, because the dataset is already curated

In [7]:
WS = re.compile(r"\s+") #if any extra whitespaces are present, we want to replace them

def clean_min(s):
   
    if not isinstance(s, str):
        return ""
        
    s = s.replace("\u00a0", " ") #replace non-breaking spaces
    s = WS.sub(" ", s)
    s = s.strip()

    return s


### Load ESCO CSV

In [10]:
# let's have a look at the ESCO dataset
df = pd.read_csv(csv_path)

print("Number of rows:", len(df))

# missing values overview

variables = []
dtypes = []
missing = []
not_missing = []

for col in df.columns:
    variables.append(col)
    dtypes.append(str(df[col].dtype))
    missing.append(df[col].isna().sum())
    not_missing.append(df[col].notna().sum())

schema_summary = pd.DataFrame({
    "variable": variables,
    "dtype": dtypes,
    "missing": missing,
    "non_missing": not_missing
})

print("ESCO schema and missing values overview:")
display(schema_summary.head(20))

#skillType overview
print("ESCO entries by skillType:")
skilltype_counts = {}

for i in range(len(df)):
    st = df.loc[i, "skillType"]
    if st not in skilltype_counts:
        skilltype_counts[st] = 0

    skilltype_counts[st] += 1

skilltype_list = []
total_rows = len(df)

for st in skilltype_counts:
    count = skilltype_counts[st]
    percentage = round(count/total_rows * 100, 2)
    skilltype_list.append([st, count, percentage])

skilltype_df = pd.DataFrame(
    skilltype_list,
    columns=["skillType", "count", "percentage"]
)

display(skilltype_df)

Number of rows: 13939
ESCO schema and missing values overview:


,variable,dtype,missing,non_missing
0,conceptType,object,0,13939
1,conceptUri,object,0,13939
2,skillType,object,5,13934
3,reuseLevel,object,5,13934
4,preferredLabel,object,1,13938
5,altLabels,object,10904,3035
6,hiddenLabels,object,13907,32
7,status,object,0,13939
8,modifiedDate,object,0,13939
9,scopeNote,object,13938,1


ESCO entries by skillType:


,skillType,count,percentage
0,skill/competence,10715,76.87
1,knowledge,3219,23.09
2,NaN,5,0.04


### Select Columns, Clean and Store as Master Table

In [13]:
label_col = "preferredLabel"
desc_col = "description"
type_col = "skillType"
id_col = "esco_id" 

if id_col not in df.columns:
    df["esco_id"] = np.arange(1, len(df) + 1)
    id_col = "esco_id"

clean_labels = []
for t in df[label_col].fillna(""):
    clean_labels.append(clean_min(str(t)))
df["label"] = clean_labels

clean_desc = []
for t in df[desc_col].fillna(""):
    clean_desc.append(clean_min(str(t)))
df["description"] = clean_desc

skill_list = []
for t in df[type_col].fillna("unknown"):
    skill_list.append(str(t))
df["skillType"] = skill_list

# we keep only the columns we will need later
df_clean = df[[id_col, "label", "description", "skillType"]].copy()
df_clean = df_clean.rename(columns={id_col: "esco_id"})

df_to_sqlite(df_clean, sqlite_out, "esco_skills")
print("Preview of cleaned ESCO skills:")
display(df_clean.head(5))



Preview of cleaned ESCO skills:


,esco_id,label,description,skillType
0,1,Musikpersonal verwalten,Zuweisen und Verwalten der Aufgaben des Person...,skill/competence
1,2,Strafvollzugsverfahren beaufsichtigen,Überwachen des Betriebs einer Justizvollzugsan...,skill/competence
2,3,nicht unterdrückende Praktiken anwenden,"Ermitteln von Repressionen in Gesellschaften, ...",skill/competence
3,4,Einhaltung von Vorschriften von Eisenbahnfahrz...,"Kontrollieren von Fahrzeugen, Komponenten und ...",skill/competence
4,5,verfügbare Dienste ermitteln,Ermitteln der verschiedenen verfügbaren Dienst...,skill/competence


### Embed and Load to SQLite
Here, two tables are created: one for labels and the second for descriptions. Each table will store embeddings in the same format as they are stored for the job data. This setup allows us to match like with like (phrase level -> labels, units -> descriptions)

In [16]:
df_labels = df_clean[["esco_id", "label", "skillType"]].copy()
df_desc = df_clean[["esco_id", "description", "skillType"]].copy()

print("Embedding labels...")

label_texts = []
for t in df_labels["label"]:
    label_texts.append(str(t))

#embed labels
label_vectors = embed_texts(label_texts)
print("Number of label embeddings:", len(label_vectors))



print("Embedding descriptions...")

desc_texts = []
for t in df_desc["description"]:
    desc_texts.append(str(t))

#embed descriptions
desc_vectors = embed_texts(desc_texts)
print("Number of description embeddings:", len(desc_vectors))


#we convert into json and store
label_json_list = []
for v in label_vectors:
    label_json_list.append(json.dumps(v, ensure_ascii=False))
df_labels["embedding_json"] = label_json_list

desc_json_list = []
for v in desc_vectors:
    desc_json_list.append(json.dumps(v, ensure_ascii=False))
df_desc["embedding_json"] = desc_json_list

print("Preview of label embeddings:")
display(df_labels.head(2))

print("Preview of description embeddings:")
display(df_desc.head(2))

Embedding labels...
Number of label embeddings: 13939
Embedding descriptions...
Number of description embeddings: 13939
Preview of label embeddings:


,esco_id,label,skillType,embedding_json
0,1,Musikpersonal verwalten,skill/competence,"[0.0632103681564331, -0.039230018854141235, 0...."
1,2,Strafvollzugsverfahren beaufsichtigen,skill/competence,"[0.00261376379057765, 0.08524057269096375, -0...."


Preview of description embeddings:


,esco_id,description,skillType,embedding_json
0,1,Zuweisen und Verwalten der Aufgaben des Person...,skill/competence,"[0.043312449008226395, -0.04002591595053673, 0..."
1,2,Überwachen des Betriebs einer Justizvollzugsan...,skill/competence,"[-0.04251798614859581, 0.03247657045722008, -0..."


In [18]:
#write the tables to the database
df_to_sqlite(df_labels, sqlite_out, "esco_labels")
df_to_sqlite(df_desc, sqlite_out, "esco_desc")

print("esco_skills size:", len(df_clean), "rows")
print("esco_labels size:", len(df_labels), "rows")
print("esco_desc size:", len(df_desc), "rows")
print("SQLite saved")


esco_skills size: 13939 rows
esco_labels size: 13939 rows
esco_desc size: 13939 rows
SQLite saved
